In [ ]:
import requests
import re
import urllib 
from bs4 import BeautifulSoup
import string
import pandas as pd
import numpy as np
import datetime
import _datetime
import xlrd
import time
import urllib.request
from selenium import webdriver


In [ ]:
current_datetime1 = datetime.datetime.now()
urlpage = 'https://stock.litopt.ru/' 
driver = webdriver.Chrome(executable_path = 'C:/Users/HYPERPC/Downloads/chromedriver')
driver.get(urlpage)

In [ ]:
driver.find_element_by_id("details-button").click()

In [ ]:
driver.find_element_by_id("proceed-link").click()

In [ ]:
pr=driver.find_elements_by_xpath("//tr[contains(concat(' ', normalize-space(@bgcolor), ' '), '#FFFFFF')]")

In [ ]:
a=[]
b=[]
c=[]
d=[]
e=[]
f=[]

In [ ]:
for j in range(len(pr)):
    #print(j)
    tyu=pr[j].find_elements_by_xpath('td')
    a.append(tyu[0].text)
    b.append(tyu[1].text)
    c.append(tyu[2].text)
    d.append(tyu[3].text)
    e.append(tyu[4].text)
    f.append(tyu[5].text)

In [ ]:
table_price = pd.DataFrame(list(zip(a,b,c,d,e,f))) 



In [ ]:
cc='C:/Users/HYPERPC/Downloads/элитпоставка.xlsx'
xl = pd.ExcelFile(cc)
df = xl.parse('TDSheet')

In [ ]:
add=[]
delete=[]

In [ ]:
for i in range(len(df)):
    if ((df['Артикул'][i] in d) and (df['Больше не поставляется'][i]=='Да')):
        add.append(df['Артикул'][i])
    else:
        if ((df['Артикул'][i] not in d) and (df['Больше не поставляется'][i]=='Нет')):
            delete.append(df['Артикул'][i])

In [ ]:
month=datetime.datetime.now().month
year=datetime.datetime.now().year
day=datetime.datetime.now().day

In [ ]:
data=str(day)+'_'+str(month)+'_'+str(year)

In [ ]:
table_price.to_excel('Прайс '+data+'.xlsx')

In [ ]:
table = pd.DataFrame(list(zip(delete)), columns =['Добавить в стоп заказ']) 

table.to_excel('Добавить в стоп заказ '+data+'.xlsx')

In [ ]:
table = pd.DataFrame(list(zip(add)), columns =['Убрать из стоп заказ']) 

table.to_excel('Убрать из стоп заказа '+data+'.xlsx')


In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

fromaddr = "agapova.a@garagetools.ru"
#toaddr = "garaeva.m@garagetools.ru"
#toaddr = "agapova.a@garagetools.ru"
toaddr = "maruschenko.a@garagetools.ru"

msg = MIMEMultipart()

msg['From'] = fromaddr
msg['To'] = toaddr
msg['Subject'] = str("Makita_"+data)

body = ""

msg.attach(MIMEText(body, 'plain'))

filename = "Ubrat_iz_stop_zakaza.xlsx"
#filename = "Уб.xlsx"
attachment = open("C:/Users/HYPERPC/Downloads/Telegram Desktop/"+"Убрать из стоп заказа "+data+".xlsx",'rb')

part = MIMEBase('application', 'octet-stream')
part.set_payload((attachment).read())
encoders.encode_base64(part)
part.add_header('Content-Disposition', "attachment; filename= %s" % filename)

msg.attach(part)

filename = "Dobavit_v_stop_zakaz.xlsx"
attachment = open("C:/Users/HYPERPC/Downloads/Telegram Desktop/"+"Добавить в стоп заказ "+data+".xlsx",'rb')

part = MIMEBase('application', 'octet-stream')
part.set_payload((attachment).read())
encoders.encode_base64(part)
part.add_header('Content-Disposition', "attachment; filename= %s" % filename)

msg.attach(part)

filename = "Price.xlsx"
attachment = open("C:/Users/HYPERPC/Downloads/Telegram Desktop/"+"Прайс "+data+".xlsx",'rb')

part = MIMEBase('application', 'octet-stream')
part.set_payload((attachment).read())
encoders.encode_base64(part)
part.add_header('Content-Disposition', "attachment; filename= %s" % filename)

msg.attach(part)



server = smtplib.SMTP('smtp.gmail.com', 587)
server.starttls()
server.login(fromaddr, "6465667Aa")
text = msg.as_string()
server.sendmail(fromaddr, toaddr, text)
server.quit()